In [ ]:
---
title: 1D unsteady heat transfer - Numerical solutions
description: Explicit method and Crank-Nicolson scheme
author: Daning H.
show-code: False
show-prompt: False
params:
    Nx:
        input: numeric
        label: Number of spatial segments
        value: 4.0
        step: 1.0
    dt:
        input: numeric
        label: Time step size
        value: 0.1
        step: 0.01
---

We solve a 1D unsteady heat transfer problem on the domain $0\leq x\leq L,\, t\geq 0$
$$
u_t = c^2u_{xx},\quad u(x,0)=f(x)
$$
where $c$ is a heat transfer constant and see figure below for the definition of initial condition $f(x)$.  Homogeneous Dirichlet problem is considered, so
$$
u(0,t) = u(L,t) = 0
$$
Here we solve the PDE using (1) an explicit scheme with central difference in space and explicit Euler in time, and (2) Crank-Nicolson scheme, i.e., an implicit scheme with central difference in space and mid-point rule in time.

Compare the accuracy and numerical stablility of the two methods by changing the sizes of spatial segments and time step.

In [1]:
Nx = 4.0
dt = 0.5

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.integrate as integrate
from IPython.display import HTML

Nx = max(int(Nx), 1)
dt = max(dt, 0.01)
Nt = max(int(10/dt), 10)

Nmod = 10
Na   = 401
xf   = 2*np.pi
cnst = 1.0
xx   = np.linspace(0, xf, Na)

def _fIC(x):
    _f = np.zeros_like(x)
    _m = x < np.pi
    _f[_m] = x[_m]
    _f[~_m] = 2*np.pi-x[~_m]
    return _f

def _anaSol(x, t):
    _s = np.zeros_like(x)
    for _i in range(Nmod):
        _k = 2*_i+1
        _s += (-1)**_i * 8/np.pi/_k**2 * np.sin(_k*xx/2) * np.exp(-(_k*cnst)**2/4*t)
    return _s

def _numSol_Exp(Nx, Nt, dt):
    _x  = np.linspace(0, xf, Nx+1)
    _f0 = _fIC(_x)
    _dx = _x[1]
    _r  = dt/_dx**2
    _sol = np.zeros((Nt+1, Nx+1))
    _sol[0] = _f0
    for _j in range(1,Nt+1):
        for _i in range(1,Nx):
            _sol[_j,_i] = _r*(_f0[_i-1] + _f0[_i+1]) + (1-2*_r)*_f0[_i]
        _f0 = _sol[_j]
    return _x, _sol

def _numSol_Imp(Nx, Nt, dt):
    _x  = np.linspace(0, xf, Nx+1)
    _f0 = _fIC(_x)
    _dx = _x[1]
    _r  = dt/_dx**2
    _fn = np.zeros_like(_x)
    _R  = _r*np.ones(Nx-1,)
    _T  = np.diag(2*(1+_R)) - np.diag(_R[:-1],k=1) - np.diag(_R[:-1],k=-1)
    _S  = np.diag(2*(1-_R)) + np.diag(_R[:-1],k=1) + np.diag(_R[:-1],k=-1)
    _A  = np.linalg.solve(_T,_S)
    _sol = np.zeros((Nt+1, Nx+1))
    _sol[0] = _f0
    for _j in range(1,Nt+1):
        _sol[_j, 1:-1] = _A.dot(_sol[_j-1, 1:-1])
    return _x, _sol

sol = [_anaSol(xx, _i*dt) for _i in range(Nt)]
_x, _fe = _numSol_Exp(Nx, Nt, dt)
_x, _fi = _numSol_Imp(Nx, Nt, dt)

fig = plt.figure()
ax = plt.gca()
ax.plot([0,np.pi,2*np.pi], [0,np.pi,0], 'k-', label='IC')
l0, = ax.plot(xx, np.zeros_like(xx), 'b-', label='Solution')
l1, = ax.plot([], [], 'ko--', label='Explicit')
l2, = ax.plot([], [], 'rs--', label='Crank-Nicolson')
ax.legend()
ax.set_xlabel('x')
ax.set_ylabel('u')
ax.set_ylim([-3.5,3.5])

def animate(i):
    l0.set_ydata(sol[i])
    l1.set_xdata(_x)
    l1.set_ydata(_fe[i])
    l2.set_xdata(_x)
    l2.set_ydata(_fi[i])
    ax.set_title('$u_t=u_{xx}$, '+f't={i*dt:4.3f}')
    return l0,

ani = animation.FuncAnimation(fig, animate, Nt, interval=250, blit=True)
plt.close(ani._fig)
HTML(ani.to_html5_video())